# Script for using linear mixed effects model to quantify differences in synaptic proteins in 5xFAD vs. WT

In [1]:
#most of the code is taken/adapted from this tutorial: https://towardsdatascience.com/how-to-run-linear-mixed-effects-models-in-python-jupyter-notebooks-4f8079c4b589

In [13]:
# Load packages
import numpy as np
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
import os

In [14]:
parentdir = '/Users/margaret/Dropbox (MIT)/mExR_analysis/SmallData/2023.05_5xFAD/'

In [15]:
os.listdir(parentdir)

['mExR_5xFAD_2023.05_reg-error_thresh99.pzfx',
 '.DS_Store',
 'archive',
 'mExR_5xFAD_abeta_cropped_alignment_notes.xlsx',
 '5xFAD_2023.05_synaptic_proteins.pzfx',
 'mExR_5xFAD_2023.05_reg-error.pzfx',
 'GluA2-4_exampes',
 'mExR_5xFAD_abeta_cropped_redo.xlsx',
 'mExR_5xFAD_abeta_cropped_GluA_D54D2_overlap.xlsx',
 '5xFAD_2023.05_synapses_WT_vs_5xFAD_RE.pzfx',
 '~$mExR_2023.05_5xFAD_notes.xlsx',
 'mExR_2023.05_5xFAD_notes.xlsx',
 '5xFAD_2023.05_abeta_WT_vs_5xFAD_RE.pzfx',
 '5xFAD_2023.05_abeta_analysis_updated.pzfx',
 '5xFAD_vs_WT_n=2_Abeta.csv',
 'mExR_5xFAD_abeta_cropped_GluA_D54D2_overlap.prism',
 '5xFAD_vs_WT_n=2_synapses.csv',
 '5xFAD_2022.05_camkiia.pzfx']

In [16]:
data = pd.read_csv(parentdir + '5xFAD_vs_WT_n=2_synapses.csv')

In [17]:
data

,Animal,ROI,Group,RIM1_vol,GluA3_vol,NR2B_vol,RIMBP_vol,GluA2_vol,GluA1_vol,NR1_vol,...,Homer1_vol,CaMKIIa_vol,Cav21_vol,GluA4_vol,PSD95_vol,Bassoon_vol,SynGAP_vol,IRsp53_vol,Stargazin_vol,Gephyrin_vol
0,5xFAD-2,ROI1,5xFAD,0.938896,0.227001,0.314918,0.696536,0.020491,0.305992,0.215331,...,0.876157,0.659500,0.534577,0.170144,0.437942,0.564158,0.365504,0.575166,0.366711,0.869217
1,5xFAD-2,ROI2,5xFAD,0.573805,0.193276,0.309231,0.411947,0.029948,0.213170,0.278290,...,0.639226,0.865067,0.428263,0.192076,0.414386,0.599792,0.362288,0.484414,0.399785,0.523541
2,5xFAD-2,ROI3,5xFAD,0.531297,0.144672,0.263887,0.277938,0.037411,0.143327,0.241540,...,0.449657,0.473105,0.379499,0.120493,0.322416,0.487506,0.291664,0.318763,0.281396,0.350383
3,5xFAD-2,ROI4,5xFAD,0.845706,0.269360,0.371717,0.618672,0.035441,0.265418,0.378900,...,0.928836,0.818258,0.261930,0.241074,0.493318,0.366737,0.439608,0.305346,0.375391,0.437882
4,5xFAD-2,ROI5,5xFAD,0.679657,0.198655,0.336618,0.545993,0.030754,0.284673,0.368149,...,0.634996,0.731348,0.479663,0.260479,0.384260,0.592225,0.403524,0.486712,0.383371,0.505465
5,5xFAD-3,ROI1,5xFAD,0.456255,0.274162,0.468582,0.589357,0.082553,0.248976,0.387343,...,0.850618,1.031245,0.684042,0.171939,0.590971,0.763008,0.580909,0.607314,0.495880,0.619017
6,5xFAD-3,ROI2,5xFAD,0.616335,0.200347,0.363775,0.468573,0.022228,0.218049,0.335157,...,0.711861,0.703539,0.503623,0.113434,0.457837,0.534148,0.453300,0.467964,0.354695,0.494030
7,5xFAD-3,ROI3,5xFAD,0.747663,0.221463,0.302661,0.344161,0.011492,0.211818,0.304672,...,0.611544,0.527297,0.392699,0.079714,0.418083,0.499670,0.337940,0.223542,0.262439,0.155753
8,5xFAD-3,ROI4,5xFAD,0.440120,0.318054,0.300684,0.417018,0.005679,0.172367,0.289781,...,0.493162,0.611746,0.396728,0.136898,0.322328,0.449585,0.321046,0.350647,0.360919,0.508065
9,WT-2,ROI1,WT,0.686920,0.352389,0.607918,0.864268,0.000711,0.387214,0.508621,...,0.902969,0.971467,0.601622,0.158117,0.803337,0.891682,0.774048,0.593516,0.591094,0.362080


In [18]:
list(data)

['Animal',
 'ROI',
 'Group',
 'RIM1_vol',
 'GluA3_vol',
 'NR2B_vol',
 'RIMBP_vol',
 'GluA2_vol',
 'GluA1_vol',
 'NR1_vol',
 'Shank3_vol',
 'Homer1_vol',
 'CaMKIIa_vol',
 'Cav21_vol',
 'GluA4_vol',
 'PSD95_vol',
 'Bassoon_vol',
 'SynGAP_vol',
 'IRsp53_vol',
 'Stargazin_vol',
 'Gephyrin_vol']

In [19]:
# Run mixed lm for RIM1 volume
md = smf.mixedlm("RIM1_vol ~ Group", data, groups=data["Animal"])
mdf = md.fit(method=["lbfgs"])
print(mdf.summary())

        Mixed Linear Model Regression Results
Model:            MixedLM Dependent Variable: RIM1_vol
No. Observations: 17      Method:             REML    
No. Groups:       4       Scale:              0.0359  
Min. group size:  4       Log-Likelihood:     -0.4910 
Max. group size:  5       Converged:          Yes     
Mean group size:  4.2                                 
------------------------------------------------------
              Coef. Std.Err.   z   P>|z| [0.025 0.975]
------------------------------------------------------
Intercept     0.641    0.179 3.586 0.000  0.290  0.991
Group[T.WT]   0.213    0.254 0.841 0.401 -0.284  0.710
Group Var     0.056    0.367                          



In [20]:
# Run mixed lm for GluA3 volume
md = smf.mixedlm("GluA3_vol ~ Group", data, groups=data["Animal"])
mdf = md.fit(method=["lbfgs"])
print(mdf.summary())

         Mixed Linear Model Regression Results
Model:            MixedLM Dependent Variable: GluA3_vol
No. Observations: 17      Method:             REML     
No. Groups:       4       Scale:              0.0023   
Min. group size:  4       Log-Likelihood:     21.8627  
Max. group size:  5       Converged:          Yes      
Mean group size:  4.2                                  
-------------------------------------------------------
              Coef. Std.Err.   z    P>|z| [0.025 0.975]
-------------------------------------------------------
Intercept     0.228    0.017 13.090 0.000  0.194  0.262
Group[T.WT]   0.121    0.025  4.801 0.000  0.071  0.170
Group Var     0.000    0.013                           



/Users/margaret/miniconda3/envs/lmer/lib/python3.9/site-packages/statsmodels/regression/mixed_linear_model.py:1635: UserWarning: Random effects covariance is singular
  warnings.warn(msg)
/Users/margaret/miniconda3/envs/lmer/lib/python3.9/site-packages/statsmodels/regression/mixed_linear_model.py:2238: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)


In [21]:
for x in range (0, 2):
    print(mdf.pvalues[x])

3.760299701479289e-39
1.576313442129688e-06


In [22]:
# Run mixed lm for NR2B volume
md = smf.mixedlm("NR2B_vol ~ Group", data, groups=data["Animal"])
mdf = md.fit(method=["lbfgs"])
print(mdf.summary())

        Mixed Linear Model Regression Results
Model:            MixedLM Dependent Variable: NR2B_vol
No. Observations: 17      Method:             REML    
No. Groups:       4       Scale:              0.0034  
Min. group size:  4       Log-Likelihood:     18.8494 
Max. group size:  5       Converged:          Yes     
Mean group size:  4.2                                 
------------------------------------------------------
             Coef. Std.Err.   z    P>|z| [0.025 0.975]
------------------------------------------------------
Intercept    0.338    0.024 14.204 0.000  0.291  0.384
Group[T.WT]  0.161    0.034  4.705 0.000  0.094  0.228
Group Var    0.000    0.022                           



/Users/margaret/miniconda3/envs/lmer/lib/python3.9/site-packages/statsmodels/regression/mixed_linear_model.py:1635: UserWarning: Random effects covariance is singular
  warnings.warn(msg)
/Users/margaret/miniconda3/envs/lmer/lib/python3.9/site-packages/statsmodels/regression/mixed_linear_model.py:2238: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)


In [23]:
for x in range (0, 2):
    print(mdf.pvalues[x])

8.621302806882892e-46
2.5360249023727947e-06


In [24]:
# Run mixed lm for RIM-BP volume
md = smf.mixedlm("RIMBP_vol ~ Group", data, groups=data["Animal"])
mdf = md.fit(method=["lbfgs"])
print(mdf.summary())

         Mixed Linear Model Regression Results
Model:            MixedLM Dependent Variable: RIMBP_vol
No. Observations: 17      Method:             REML     
No. Groups:       4       Scale:              0.0131   
Min. group size:  4       Log-Likelihood:     8.1561   
Max. group size:  5       Converged:          Yes      
Mean group size:  4.2                                  
-------------------------------------------------------
               Coef. Std.Err.   z   P>|z| [0.025 0.975]
-------------------------------------------------------
Intercept      0.484    0.062 7.796 0.000  0.362  0.605
Group[T.WT]    0.199    0.089 2.245 0.025  0.025  0.373
Group Var      0.005    0.075                          



/Users/margaret/miniconda3/envs/lmer/lib/python3.9/site-packages/statsmodels/regression/mixed_linear_model.py:1635: UserWarning: Random effects covariance is singular
  warnings.warn(msg)
/Users/margaret/miniconda3/envs/lmer/lib/python3.9/site-packages/statsmodels/regression/mixed_linear_model.py:2238: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)


In [25]:
for x in range (0, 2):
    print(mdf.pvalues[x])

6.392175851209788e-15
0.024796891084460697


In [26]:
# Run mixed lm for GluA2 volume
md = smf.mixedlm("GluA2_vol ~ Group", data, groups=data["Animal"])
mdf = md.fit(method=["lbfgs"])
print(mdf.summary())

         Mixed Linear Model Regression Results
Model:            MixedLM Dependent Variable: GluA2_vol
No. Observations: 17      Method:             REML     
No. Groups:       4       Scale:              0.0003   
Min. group size:  4       Log-Likelihood:     38.3937  
Max. group size:  5       Converged:          Yes      
Mean group size:  4.2                                  
-------------------------------------------------------
             Coef.  Std.Err.   z    P>|z| [0.025 0.975]
-------------------------------------------------------
Intercept     0.031    0.005  5.630 0.000  0.020  0.041
Group[T.WT]  -0.029    0.008 -3.622 0.000 -0.044 -0.013
Group Var     0.000                                    



/Users/margaret/miniconda3/envs/lmer/lib/python3.9/site-packages/statsmodels/regression/mixed_linear_model.py:1635: UserWarning: Random effects covariance is singular
  warnings.warn(msg)
/Users/margaret/miniconda3/envs/lmer/lib/python3.9/site-packages/statsmodels/regression/mixed_linear_model.py:2238: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)
/Users/margaret/miniconda3/envs/lmer/lib/python3.9/site-packages/statsmodels/regression/mixed_linear_model.py:2262: ConvergenceWarning: The Hessian matrix at the estimated parameter values is not positive definite.
  warnings.warn(msg, ConvergenceWarning)


In [27]:
for x in range (0, 2):
    print(mdf.pvalues[x])

1.805452044122285e-08
0.0002923660533702865


In [28]:
# Run mixed lm for GluA1 volume
md = smf.mixedlm("GluA1_vol ~ Group", data, groups=data["Animal"])
mdf = md.fit(method=["lbfgs"])
print(mdf.summary())

         Mixed Linear Model Regression Results
Model:            MixedLM Dependent Variable: GluA1_vol
No. Observations: 17      Method:             REML     
No. Groups:       4       Scale:              0.0019   
Min. group size:  4       Log-Likelihood:     23.0466  
Max. group size:  5       Converged:          No       
Mean group size:  4.2                                  
-------------------------------------------------------
              Coef. Std.Err.   z    P>|z| [0.025 0.975]
-------------------------------------------------------
Intercept     0.229    0.019 12.202 0.000  0.192  0.265
Group[T.WT]   0.127    0.027  4.713 0.000  0.074  0.180
Group Var     0.000    0.024                           



/Users/margaret/miniconda3/envs/lmer/lib/python3.9/site-packages/statsmodels/regression/mixed_linear_model.py:1635: UserWarning: Random effects covariance is singular
  warnings.warn(msg)
/Users/margaret/miniconda3/envs/lmer/lib/python3.9/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/Users/margaret/miniconda3/envs/lmer/lib/python3.9/site-packages/statsmodels/regression/mixed_linear_model.py:2207: ConvergenceWarning: MixedLM optimization failed, trying a different optimizer may help.
  warnings.warn(msg, ConvergenceWarning)
/Users/margaret/miniconda3/envs/lmer/lib/python3.9/site-packages/statsmodels/regression/mixed_linear_model.py:2219: ConvergenceWarning: Gradient optimization failed, |grad| = 0.430726
  warnings.warn(msg, ConvergenceWarning)
/Users/margaret/miniconda3/envs/lmer/lib/python3.9/site-packages/statsmodels/regression/mixed_l

In [29]:
for x in range (0, 2):
    print(mdf.pvalues[x])

3.0443632292401968e-34
2.4387804893124456e-06


In [30]:
# Run mixed lm for NR1 volume
md = smf.mixedlm("NR1_vol ~ Group", data, groups=data["Animal"])
mdf = md.fit(method=["lbfgs"])
print(mdf.summary())

        Mixed Linear Model Regression Results
Model:            MixedLM Dependent Variable: NR1_vol
No. Observations: 17      Method:             REML   
No. Groups:       4       Scale:              0.0034 
Min. group size:  4       Log-Likelihood:     19.1382
Max. group size:  5       Converged:          Yes    
Mean group size:  4.2                                
-----------------------------------------------------
            Coef. Std.Err.   z    P>|z| [0.025 0.975]
-----------------------------------------------------
Intercept   0.311    0.021 14.973 0.000  0.270  0.352
Group[T.WT] 0.142    0.029  4.836 0.000  0.084  0.199
Group Var   0.000    0.051                           



/Users/margaret/miniconda3/envs/lmer/lib/python3.9/site-packages/statsmodels/regression/mixed_linear_model.py:1635: UserWarning: Random effects covariance is singular
  warnings.warn(msg)
/Users/margaret/miniconda3/envs/lmer/lib/python3.9/site-packages/statsmodels/regression/mixed_linear_model.py:2238: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)


In [31]:
for x in range (0, 2):
    print(mdf.pvalues[x])

1.0987047905077468e-50
1.3268324918232215e-06


In [32]:
# Run mixed lm for Shank3 volume
md = smf.mixedlm("Shank3_vol ~ Group", data, groups=data["Animal"])
mdf = md.fit(method=["lbfgs"])
print(mdf.summary())

         Mixed Linear Model Regression Results
Model:            MixedLM Dependent Variable: Shank3_vol
No. Observations: 17      Method:             REML      
No. Groups:       4       Scale:              0.0044    
Min. group size:  4       Log-Likelihood:     15.7101   
Max. group size:  5       Converged:          Yes       
Mean group size:  4.2                                   
--------------------------------------------------------
                Coef. Std.Err.   z   P>|z| [0.025 0.975]
--------------------------------------------------------
Intercept       0.434    0.051 8.501 0.000  0.334  0.534
Group[T.WT]     0.248    0.073 3.412 0.001  0.105  0.390
Group Var       0.004    0.087                          



/Users/margaret/miniconda3/envs/lmer/lib/python3.9/site-packages/statsmodels/regression/mixed_linear_model.py:1635: UserWarning: Random effects covariance is singular
  warnings.warn(msg)
/Users/margaret/miniconda3/envs/lmer/lib/python3.9/site-packages/statsmodels/regression/mixed_linear_model.py:2238: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)


In [33]:
for x in range (0, 2):
    print(mdf.pvalues[x])

1.876848449818151e-17
0.0006437639098512915


In [34]:
# Run mixed lm for Homer1 volume
md = smf.mixedlm("Homer1_vol ~ Group", data, groups=data["Animal"])
mdf = md.fit(method=["lbfgs"])
print(mdf.summary())

         Mixed Linear Model Regression Results
Model:            MixedLM Dependent Variable: Homer1_vol
No. Observations: 17      Method:             REML      
No. Groups:       4       Scale:              0.0188    
Min. group size:  4       Log-Likelihood:     6.3876    
Max. group size:  5       Converged:          Yes       
Mean group size:  4.2                                   
--------------------------------------------------------
               Coef. Std.Err.   z    P>|z| [0.025 0.975]
--------------------------------------------------------
Intercept      0.688    0.046 15.027 0.000  0.599  0.778
Group[T.WT]    0.124    0.067  1.865 0.062 -0.006  0.255
Group Var      0.000    0.049                           



/Users/margaret/miniconda3/envs/lmer/lib/python3.9/site-packages/statsmodels/regression/mixed_linear_model.py:1635: UserWarning: Random effects covariance is singular
  warnings.warn(msg)
/Users/margaret/miniconda3/envs/lmer/lib/python3.9/site-packages/statsmodels/regression/mixed_linear_model.py:2238: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)


In [35]:
for x in range (0, 2):
    print(mdf.pvalues[x])

4.890826724293538e-51
0.06213637874930563


In [36]:
list(data)

['Animal',
 'ROI',
 'Group',
 'RIM1_vol',
 'GluA3_vol',
 'NR2B_vol',
 'RIMBP_vol',
 'GluA2_vol',
 'GluA1_vol',
 'NR1_vol',
 'Shank3_vol',
 'Homer1_vol',
 'CaMKIIa_vol',
 'Cav21_vol',
 'GluA4_vol',
 'PSD95_vol',
 'Bassoon_vol',
 'SynGAP_vol',
 'IRsp53_vol',
 'Stargazin_vol',
 'Gephyrin_vol']

In [37]:
# Run mixed lm for CaMKIIa
md = smf.mixedlm("CaMKIIa_vol ~ Group", data, groups=data["Animal"])
mdf = md.fit(method=["lbfgs"])
print(mdf.summary())

          Mixed Linear Model Regression Results
Model:            MixedLM Dependent Variable: CaMKIIa_vol
No. Observations: 17      Method:             REML       
No. Groups:       4       Scale:              0.0272     
Min. group size:  4       Log-Likelihood:     3.0523     
Max. group size:  5       Converged:          No         
Mean group size:  4.2                                    
---------------------------------------------------------
               Coef.  Std.Err.   z    P>|z| [0.025 0.975]
---------------------------------------------------------
Intercept       0.714    0.074  9.687 0.000  0.569  0.858
Group[T.WT]    -0.057    0.106 -0.541 0.588 -0.265  0.150
Group Var       0.005    0.121                           



/Users/margaret/miniconda3/envs/lmer/lib/python3.9/site-packages/statsmodels/regression/mixed_linear_model.py:1635: UserWarning: Random effects covariance is singular
  warnings.warn(msg)
/Users/margaret/miniconda3/envs/lmer/lib/python3.9/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/Users/margaret/miniconda3/envs/lmer/lib/python3.9/site-packages/statsmodels/regression/mixed_linear_model.py:2207: ConvergenceWarning: MixedLM optimization failed, trying a different optimizer may help.
  warnings.warn(msg, ConvergenceWarning)
/Users/margaret/miniconda3/envs/lmer/lib/python3.9/site-packages/statsmodels/regression/mixed_linear_model.py:2219: ConvergenceWarning: Gradient optimization failed, |grad| = 0.581352
  warnings.warn(msg, ConvergenceWarning)
/Users/margaret/miniconda3/envs/lmer/lib/python3.9/site-packages/statsmodels/regression/mixed_l

In [38]:
for x in range (0, 2):
    print(mdf.pvalues[x])

3.441134629528222e-22
0.5884591077447816


In [39]:
# Run mixed lm for Cav2.1
md = smf.mixedlm("Cav21_vol ~ Group", data, groups=data["Animal"])
mdf = md.fit(method=["lbfgs"])
print(mdf.summary())

         Mixed Linear Model Regression Results
Model:            MixedLM Dependent Variable: Cav21_vol
No. Observations: 17      Method:             REML     
No. Groups:       4       Scale:              0.0091   
Min. group size:  4       Log-Likelihood:     11.6582  
Max. group size:  5       Converged:          Yes      
Mean group size:  4.2                                  
-------------------------------------------------------
              Coef. Std.Err.   z    P>|z| [0.025 0.975]
-------------------------------------------------------
Intercept     0.452    0.035 13.025 0.000  0.384  0.520
Group[T.WT]   0.155    0.050  3.098 0.002  0.057  0.254
Group Var     0.000    0.028                           



/Users/margaret/miniconda3/envs/lmer/lib/python3.9/site-packages/statsmodels/regression/mixed_linear_model.py:1635: UserWarning: Random effects covariance is singular
  warnings.warn(msg)
/Users/margaret/miniconda3/envs/lmer/lib/python3.9/site-packages/statsmodels/regression/mixed_linear_model.py:2238: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)


In [40]:
for x in range (0, 2):
    print(mdf.pvalues[x])

8.872830899680969e-39
0.001950926564825465


In [41]:
# Run mixed lm for GluA4 volume
md = smf.mixedlm("GluA4_vol ~ Group", data, groups=data["Animal"])
mdf = md.fit(method=["lbfgs"])
print(mdf.summary())

         Mixed Linear Model Regression Results
Model:            MixedLM Dependent Variable: GluA4_vol
No. Observations: 17      Method:             REML     
No. Groups:       4       Scale:              0.0017   
Min. group size:  4       Log-Likelihood:     23.2332  
Max. group size:  5       Converged:          Yes      
Mean group size:  4.2                                  
-------------------------------------------------------
             Coef.  Std.Err.   z    P>|z| [0.025 0.975]
-------------------------------------------------------
Intercept     0.162    0.026  6.346 0.000  0.112  0.212
Group[T.WT]  -0.052    0.036 -1.438 0.150 -0.124  0.019
Group Var     0.001    0.034                           



/Users/margaret/miniconda3/envs/lmer/lib/python3.9/site-packages/statsmodels/regression/mixed_linear_model.py:1635: UserWarning: Random effects covariance is singular
  warnings.warn(msg)
/Users/margaret/miniconda3/envs/lmer/lib/python3.9/site-packages/statsmodels/regression/mixed_linear_model.py:2238: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)


In [42]:
for x in range (0, 2):
    print(mdf.pvalues[x])

2.2091331620613374e-10
0.15040400452479294


In [43]:
# Run mixed lm for PSD95 volume
md = smf.mixedlm("PSD95_vol ~ Group", data, groups=data["Animal"])
mdf = md.fit(method=["lbfgs"])
print(mdf.summary())

         Mixed Linear Model Regression Results
Model:            MixedLM Dependent Variable: PSD95_vol
No. Observations: 17      Method:             REML     
No. Groups:       4       Scale:              0.0052   
Min. group size:  4       Log-Likelihood:     14.0913  
Max. group size:  5       Converged:          Yes      
Mean group size:  4.2                                  
-------------------------------------------------------
               Coef. Std.Err.   z   P>|z| [0.025 0.975]
-------------------------------------------------------
Intercept      0.429    0.066 6.524 0.000  0.300  0.557
Group[T.WT]    0.321    0.093 3.441 0.001  0.138  0.504
Group Var      0.007    0.131                          



/Users/margaret/miniconda3/envs/lmer/lib/python3.9/site-packages/statsmodels/regression/mixed_linear_model.py:2238: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)


In [44]:
for x in range (0, 2):
    print(mdf.pvalues[x])

6.85350012744784e-11
0.0005802822954052866


In [45]:
# Run mixed lm for Bassoon volume
md = smf.mixedlm("Bassoon_vol ~ Group", data, groups=data["Animal"])
mdf = md.fit(method=["lbfgs"])
print(mdf.summary())

          Mixed Linear Model Regression Results
Model:            MixedLM Dependent Variable: Bassoon_vol
No. Observations: 17      Method:             REML       
No. Groups:       4       Scale:              0.0105     
Min. group size:  4       Log-Likelihood:     10.4959    
Max. group size:  5       Converged:          No         
Mean group size:  4.2                                    
---------------------------------------------------------
                Coef. Std.Err.   z    P>|z| [0.025 0.975]
---------------------------------------------------------
Intercept       0.540    0.039 13.832 0.000  0.464  0.617
Group[T.WT]     0.307    0.056  5.431 0.000  0.196  0.417
Group Var       0.001    0.036                           



/Users/margaret/miniconda3/envs/lmer/lib/python3.9/site-packages/statsmodels/regression/mixed_linear_model.py:1635: UserWarning: Random effects covariance is singular
  warnings.warn(msg)
/Users/margaret/miniconda3/envs/lmer/lib/python3.9/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/Users/margaret/miniconda3/envs/lmer/lib/python3.9/site-packages/statsmodels/regression/mixed_linear_model.py:2207: ConvergenceWarning: MixedLM optimization failed, trying a different optimizer may help.
  warnings.warn(msg, ConvergenceWarning)
/Users/margaret/miniconda3/envs/lmer/lib/python3.9/site-packages/statsmodels/regression/mixed_linear_model.py:2219: ConvergenceWarning: Gradient optimization failed, |grad| = 0.046479
  warnings.warn(msg, ConvergenceWarning)
/Users/margaret/miniconda3/envs/lmer/lib/python3.9/site-packages/statsmodels/regression/mixed_l

In [46]:
for x in range (0, 2):
    print(mdf.pvalues[x])

1.6242738300211687e-43
5.615289489075852e-08


In [47]:
# Run mixed lm for SynGAP volume
md = smf.mixedlm("SynGAP_vol ~ Group", data, groups=data["Animal"])
mdf = md.fit(method=["lbfgs"])
print(mdf.summary())

         Mixed Linear Model Regression Results
Model:            MixedLM Dependent Variable: SynGAP_vol
No. Observations: 17      Method:             REML      
No. Groups:       4       Scale:              0.0080    
Min. group size:  4       Log-Likelihood:     12.8048   
Max. group size:  5       Converged:          Yes       
Mean group size:  4.2                                   
--------------------------------------------------------
               Coef. Std.Err.   z    P>|z| [0.025 0.975]
--------------------------------------------------------
Intercept      0.395    0.030 13.168 0.000  0.336  0.454
Group[T.WT]    0.336    0.044  7.699 0.000  0.250  0.421
Group Var      0.000    0.023                           



/Users/margaret/miniconda3/envs/lmer/lib/python3.9/site-packages/statsmodels/regression/mixed_linear_model.py:1635: UserWarning: Random effects covariance is singular
  warnings.warn(msg)
/Users/margaret/miniconda3/envs/lmer/lib/python3.9/site-packages/statsmodels/regression/mixed_linear_model.py:2238: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)


In [48]:
for x in range (0, 2):
    print(mdf.pvalues[x])

1.34048130806913e-39
1.3722557373037156e-14


In [49]:
# Run mixed lm for IRsp53 volume
md = smf.mixedlm("IRsp53_vol ~ Group", data, groups=data["Animal"])
mdf = md.fit(method=["lbfgs"])
print(mdf.summary())

         Mixed Linear Model Regression Results
Model:            MixedLM Dependent Variable: IRsp53_vol
No. Observations: 17      Method:             REML      
No. Groups:       4       Scale:              0.0116    
Min. group size:  4       Log-Likelihood:     9.9748    
Max. group size:  5       Converged:          No        
Mean group size:  4.2                                   
--------------------------------------------------------
               Coef. Std.Err.   z    P>|z| [0.025 0.975]
--------------------------------------------------------
Intercept      0.424    0.036 11.808 0.000  0.354  0.495
Group[T.WT]    0.097    0.052  1.851 0.064 -0.006  0.200
Group Var      0.000                                    



/Users/margaret/miniconda3/envs/lmer/lib/python3.9/site-packages/statsmodels/regression/mixed_linear_model.py:1635: UserWarning: Random effects covariance is singular
  warnings.warn(msg)
/Users/margaret/miniconda3/envs/lmer/lib/python3.9/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/Users/margaret/miniconda3/envs/lmer/lib/python3.9/site-packages/statsmodels/regression/mixed_linear_model.py:2207: ConvergenceWarning: MixedLM optimization failed, trying a different optimizer may help.
  warnings.warn(msg, ConvergenceWarning)
/Users/margaret/miniconda3/envs/lmer/lib/python3.9/site-packages/statsmodels/regression/mixed_linear_model.py:2219: ConvergenceWarning: Gradient optimization failed, |grad| = 0.058529
  warnings.warn(msg, ConvergenceWarning)
/Users/margaret/miniconda3/envs/lmer/lib/python3.9/site-packages/statsmodels/regression/mixed_l

In [50]:
for x in range (0, 2):
    print(mdf.pvalues[x])

3.549026936036346e-32
0.06418913305294463


In [51]:
list(data)

['Animal',
 'ROI',
 'Group',
 'RIM1_vol',
 'GluA3_vol',
 'NR2B_vol',
 'RIMBP_vol',
 'GluA2_vol',
 'GluA1_vol',
 'NR1_vol',
 'Shank3_vol',
 'Homer1_vol',
 'CaMKIIa_vol',
 'Cav21_vol',
 'GluA4_vol',
 'PSD95_vol',
 'Bassoon_vol',
 'SynGAP_vol',
 'IRsp53_vol',
 'Stargazin_vol',
 'Gephyrin_vol']

In [52]:
# Run mixed lm for Stargazin
md = smf.mixedlm("Stargazin_vol ~ Group", data, groups=data["Animal"])
mdf = md.fit(method=["lbfgs"])
print(mdf.summary())

           Mixed Linear Model Regression Results
Model:            MixedLM Dependent Variable: Stargazin_vol
No. Observations: 17      Method:             REML         
No. Groups:       4       Scale:              0.0041       
Min. group size:  4       Log-Likelihood:     16.1458      
Max. group size:  5       Converged:          Yes          
Mean group size:  4.2                                      
------------------------------------------------------------
               Coef.  Std.Err.    z    P>|z|  [0.025  0.975]
------------------------------------------------------------
Intercept      0.365     0.049  7.394  0.000   0.268   0.462
Group[T.WT]    0.096     0.070  1.363  0.173  -0.042   0.233
Group Var      0.004     0.083                              



/Users/margaret/miniconda3/envs/lmer/lib/python3.9/site-packages/statsmodels/regression/mixed_linear_model.py:1635: UserWarning: Random effects covariance is singular
  warnings.warn(msg)
/Users/margaret/miniconda3/envs/lmer/lib/python3.9/site-packages/statsmodels/regression/mixed_linear_model.py:2238: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)


In [53]:
for x in range (0, 2):
    print(mdf.pvalues[x])

1.4232786878272134e-13
0.17295562073660908


In [54]:
# Run mixed lm for Gephyrin
md = smf.mixedlm("Gephyrin_vol ~ Group", data, groups=data["Animal"])
mdf = md.fit(method=["lbfgs"])
print(mdf.summary())

          Mixed Linear Model Regression Results
Model:            MixedLM Dependent Variable: Gephyrin_vol
No. Observations: 17      Method:             REML        
No. Groups:       4       Scale:              0.0240      
Min. group size:  4       Log-Likelihood:     4.4330      
Max. group size:  5       Converged:          Yes         
Mean group size:  4.2                                     
----------------------------------------------------------
                Coef.  Std.Err.   z    P>|z| [0.025 0.975]
----------------------------------------------------------
Intercept        0.495    0.055  9.030 0.000  0.388  0.603
Group[T.WT]     -0.156    0.080 -1.967 0.049 -0.312 -0.001
Group Var        0.001    0.044                           



/Users/margaret/miniconda3/envs/lmer/lib/python3.9/site-packages/statsmodels/regression/mixed_linear_model.py:1635: UserWarning: Random effects covariance is singular
  warnings.warn(msg)
/Users/margaret/miniconda3/envs/lmer/lib/python3.9/site-packages/statsmodels/regression/mixed_linear_model.py:2238: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)


In [55]:
for x in range (0, 2):
    print(mdf.pvalues[x])

1.7191233508555876e-19
0.049220121489328736
